In [1]:
# Begin by cloning the development branch of MAST-ML to the Colab session
!git clone --single-branch --branch dev_Ryan_2020-12-21 https://github.com/uw-cmg/MAST-ML

Cloning into 'MAST-ML'...
remote: Enumerating objects: 315, done.
remote: Counting objects: 100% (315/315), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 18515 (delta 261), reused 278 (delta 244), pack-reused 18200
Receiving objects: 100% (18515/18515), 131.36 MiB | 22.92 MiB/s, done.
Resolving deltas: 100% (12322/12322), done.


In [2]:
# Next, we install the required dependencies of MAST-ML to our Colab session
!pip install -r MAST-ML/requirements.txt

     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 92kB 5.3MB/s 
     |████████████████████████████████| 5.8MB 7.7MB/s 
     |████████████████████████████████| 3.0MB 48.8MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 102kB 11.8MB/s 
     |████████████████████████████████| 81kB 10.9MB/s 
     |████████████████████████████████| 645kB 54.5MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
     |████████████████████████████████| 81kB 11.2MB/s 
     |████████████████████████████████| 215kB 50.4MB/s 
     |████████████████████████████████| 13.2MB 52.9MB/s 
     |████████████████████████████████| 829kB 48.2MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
     |████████████████████████████████| 256kB 50.1MB/s 
   

In [3]:
# Sync your Google drive to Colab so that we can save MAST-ML results to our Google
# Drive. If we save to the Colab session, the data will be deleted when the session 
# ends.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
# We need to add the MAST-ML folder to our sys path so that python can find the modules
import sys
sys.path.append('MAST-ML')

In [5]:
# Here we import the MAST-ML modules used in this tutorial
from mastml.mastml import Mastml
from mastml.datasets import LocalDatasets, DataCleaning
from mastml.preprocessing import SklearnPreprocessor
from mastml.models import SklearnModel
from mastml.data_splitters import SklearnDataSplitter, NoSplit
from mastml.feature_selectors import EnsembleModelFeatureSelector, NoSelect

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


To install latest forestci compatabilty with scikit-learn>=0.24, run pip install git+git://github.com/scikit-learn-contrib/forest-confidence-interval.git
To import data from figshare, manually install figshare via git clone of git clone https://github.com/cognoma/figshare.git


In [6]:
# Set the name of the savepath to save MAST-ML results to
SAVEPATH = 'drive/MyDrive/MASTML_results_2021-03-01'

# Initialize the MAST-ML run, write savepath
mastml = Mastml(savepath=SAVEPATH)
savepath = mastml.get_savepath

drive/MyDrive/MASTML_results_2021-03-01 not empty. Renaming...


In [7]:
# Here, we use the LocalDatasets module to load in a dataset. We are using the
# diffusion dataset for this example

# Need to denote the column name of the target (y-data)
target = 'E_regression.1'

# There are columns in the data file not used as features or target. We need to
# list them here in the parameter extra_columns
extra_columns = ['Material compositions 1', 'Material compositions 2', 'Hop activation barrier', 'E_regression']

# Here, we make an instance of our LocalDatasets class. It needs a few parameters:
#   file_path: where the data is stored
#   target: the column name of the y-data
#   extra_columns: list containing extra columns in the data file not used for fitting
#   group_column: column name denoting group labels (only used for LeaveOutGroup CV)
#   testdata_columns: column names denoting left-out data to evaluate using best
#     model from CV tests. This is manual way to leave out data. Can also be done
#     automatically using nested CV (will see below)
#   as_frame: whether to return data as dataframe. Want this to be true.
d = LocalDatasets(file_path='MAST-ML/figshare_7418492/All_Model_Data.xlsx', 
                  target=target, 
                  extra_columns=extra_columns, 
                  group_column='Material compositions 1',
                  testdata_columns=None,
                  as_frame=True)

# Load the data with the load_data() method
data_dict = d.load_data()

In [8]:
# Let's look at the contents of the loaded data_dict
data_dict.keys()

# We see there are 5 keys:
#   X: the X feature matrix (used to fit the ML model)
#   y: the y target data vector (true values)
#   X_extra: matrix of extra information not used in fitting (i.e. not part of X or y)
#   groups: vector of group labels (here, a list of host elements)
#   X_testdata: matrix or vector of left out data. Empty for our current example.

dict_keys(['X', 'y', 'groups', 'X_extra', 'X_testdata'])

In [9]:
# Let's assign each data object to its respective name

X = data_dict['X']
y = data_dict['y']
X_extra = data_dict['X_extra']
groups = data_dict['groups']
X_testdata = data_dict['X_testdata']

In [10]:
# Let's have a look at the X feature matrix. It contains 287 elemental features.
# There are a total of 408 diffusion activation barriers used for fitting
X

,valence_composition_average,phi_composition_average,NdValence_composition_average,MiracleRadius_composition_average,GSestFCClatcnt_composition_average,SecondIonizationEnergy_composition_average,IonizationEnergy_composition_average,n_ws^third_composition_average,ThermalConductivity_composition_average,CovalentRadius_composition_average,Column_composition_average,NsValence_composition_average,NUnfilled_composition_average,CovalentRadii_composition_average,Density_composition_average,BCCenergy_pa_composition_average,BCCvolume_padiff_composition_average,BCCvolume_pa_composition_average,MeltingT_composition_average,SpaceGroupNumber_composition_average,NValance_composition_average,ElectricalConductivity_composition_average,FirstIonizationEnergy_composition_average,GSvolume_pa_composition_average,Row_composition_average,Polarizability_composition_average,AtomicNumber_composition_average,ElectronAffinity_composition_average,BCCfermi_composition_average,GSestBCClatcnt_composition_average,HHIr_composition_average,HHIp_composition_average,GSenergy_pa_composition_average,ThirdIonizationEnergy_composition_average,BulkModulus_composition_average,BoilingT_composition_average,ICSDVolume_composition_average,HeatVaporization_composition_average,IonicRadii_composition_average,NdUnfilled_composition_average,...,Site2_NdValence,Site2_MiracleRadius,Site2_GSestFCClatcnt,Site2_IonizationEnergy,Site2_ThermalConductivity,Site2_CovalentRadius,Site2_Column,Site2_NUnfilled,Site2_CovalentRadii,Site2_Density,Site2_BCCenergy_pa,Site2_BCCvolume_padiff,Site2_BCCvolume_pa,Site2_MeltingT,Site2_SpaceGroupNumber,Site2_NValance,Site2_ElectricalConductivity,Site2_FirstIonizationEnergy,Site2_GSvolume_pa,Site2_Row,Site2_Polarizability,Site2_AtomicNumber,Site2_BCCfermi,Site2_GSestBCClatcnt,Site2_GSenergy_pa,Site2_BoilingT,Site2_ICSDVolume,Site2_HeatVaporization,Site2_IonicRadii,Site2_NdUnfilled,Site2_MendeleevNumber,Site2_ElasticModulus,Site2_Electronegativity,Site2_AtomicWeight,Site2_HeatFusion,Site2_SpecificHeatCapacity,Site2_AtomicRadii,Site2_ThermalExpansionCoefficient,Site2_BCCefflatcnt,Site2_AtomicVolume
0,2.0,4.350,10.0,144.0,4.027313,21.4900,731.00,1.360,429.00,145.0,11.0,1.0,1.0,1.340,10490.0,-2.719979,-0.1300,16.200,1234.930,225.0,11.0,62.90,7.576230,16.3300,5.0,6.990,47.0,125.70,4.052760,3.196481,1400,1200,-2.765394,34.8300,100.0,2435.0,17.00,250.630,1.150,0.0,...,10,144,4.027313,731.0,429.0,145,11,1,1.34,10490,-2.719979,-0.130,16.20,1234.93,225,11,62.9,7.57623,16.330,5,6.99,47,4.052760,3.196481,-2.765394,2435.0,17.0,250.63,1.15,0,65,80.0,1.93,107.868200,11.30,0.235,1.444,18.9,6.375951,17.075648
1,2.5,4.725,8.5,134.5,3.737485,19.2750,744.50,1.555,264.50,135.5,10.0,1.5,2.0,1.250,9695.0,-4.850033,0.0325,13.320,1501.465,209.5,10.0,40.40,7.728620,13.2875,4.5,7.245,37.0,97.85,5.096587,2.966444,2050,2150,-4.922891,34.1650,140.0,2817.5,14.05,311.965,0.900,1.5,...,7,125,3.447656,758.0,100.0,126,9,3,1.16,8900,-6.980087,0.195,10.44,1768.00,194,9,17.9,7.88101,10.245,4,7.50,27,6.140415,2.736407,-7.080389,3200.0,11.1,373.30,0.65,3,58,208.0,1.88,58.933195,16.20,0.421,1.253,13.0,5.507318,10.995861
2,4.0,4.500,7.5,137.0,3.788936,18.9950,691.90,1.545,261.35,142.0,8.5,1.0,3.5,1.260,8815.0,-6.112393,-0.0750,13.685,1707.465,227.0,8.5,35.40,7.171370,13.7600,4.5,9.295,35.5,94.85,6.014901,3.007280,2750,2150,-6.135555,32.8950,130.0,2689.5,14.50,295.065,0.885,2.5,...,5,130,3.550559,652.8,93.7,139,6,6,1.18,7140,-9.504808,-0.020,11.17,2180.00,229,6,7.9,6.76651,11.190,4,11.60,24,7.977043,2.818080,-9.505717,2944.0,12.0,339.50,0.62,5,49,259.0,1.66,51.996100,21.00,0.449,1.249,4.9,5.632801,12.092937
3,2.0,4.400,10.0,135.0,3.782567,20.8910,738.20,1.415,415.00,138.5,11.0,1.0,1.0,1.255,9705.0,-3.181433,-0.0600,13.640,1296.350,225.0,11.0,61.80,7.651305,13.7000,4.5,6.570,38.0,122.00,4.215750,3.002225,1450,1400,-3.223024,35.8300,120.0,2817.5,14.40,275.565,0.940,0.0,...,10,126,3.537821,745.4,401.0,132,11,1,1.17,8920,-3.642887,0.010,11.08,1357.77,225,11,60.7,7.72638,11.070,4,6.15,29,4.378740,2.80

In [11]:
# Let's look at the groups list. The groups denote the element identity of the host 
# metal
groups

0      Ag
1      Ag
2      Ag
3      Ag
4      Ag
       ..
403    Zr
404    Zr
405    Zr
406    Zr
407    Zr
Name: Material compositions 1, Length: 408, dtype: object

In [12]:
# Let's examine the extra data not used in fitting. Note that the y-data we are 
# fitting to are equal to the E_regression values listed here, minus the E_regression
# value of the pure material. Material compositions 1 are the host elements, which
# are the same values used in the groups list. Material compositions 2 are the solute
# elements (i.e. the diffusing impurity in the host material)
X_extra

,Material compositions 1,Material compositions 2,Hop activation barrier,E_regression
0,Ag,Ag,-0.365590,1.824500
1,Ag,Co,-0.675263,1.734358
2,Ag,Cr,-0.047690,2.083639
3,Ag,Cu,-0.428459,1.802300
4,Ag,Fe,0.081529,2.142172
...,...,...,...,...
403,Zr,Nb,-0.478007,2.523770
404,Zr,Ta,-0.054726,2.747650
405,Zr,Ti,-0.022476,2.842630
406,Zr,Hf,-0.104201,2.801830


In [13]:
# Finally, we can look at our manually left-out data set. For this tutorial we
# have left this blank
X_testdata

In [14]:
# If the data contains missing values (this one doesn't), we can clean the data, 
# which corrects missing values and provides some basic analysis of the input data.
# Since there are no missing values the data cleaner will still output some useful
# plots and statistics of our input data

cleaner = DataCleaning()
X, y = cleaner.evaluate(X=X, 
                        y=y, 
                        method='imputation', 
                        strategy='mean', 
                        savepath=savepath)

In [15]:
# Here is where we can specify some of the core MAST-ML options, like how to 
# preprocess our data, what models to use in fitting, and how to select features.
# In addition, we would also specify how to optimize model hyperparameters, but
# we don't need to do that for this first tutorial. Lastly, we can decide which
# metrics to evaluate in our fits.

# Here, we define two models. The first is a random forest model, the second is
# a Gaussian process model. The scikit-learn model name can just be given as
# a string matching the model name in the "model" field. The remaining arguments
# are the parameters to pass to the model. If no parameters are given, default
# values are used.
model1 = SklearnModel(model='RandomForestRegressor', n_estimators=150, max_depth=30)
model2 = SklearnModel(model='GaussianProcessRegressor', kernel='ConstantKernel*RBF', n_restarts_optimizer=10)
# MAST-ML takes a list of the models as input.
models = [model1, model2]

# Here, we define our preprocessing function. We are just going to use the basic
# StandardScaler in scikit-learn to normalize each column to have mean zero and
# standard deviation of one.
preprocessor = SklearnPreprocessor(preprocessor='StandardScaler', as_frame=True)

# Next, we define a couple ways to select features. The first method, NoSelect,
# just uses all features in the X feature set. The second method, EnsembleModelFeatureSelector,
# uses a random forest model to select the top (here, 20) features based on 
# ranked random forest feature importance scores.
selector1 = NoSelect()
selector2 = EnsembleModelFeatureSelector(model=SklearnModel(model='RandomForestRegressor'), n_features_to_select=20)
selectors = [selector1, selector2]

# Finally, we list which metrics we want to evaluate. If none are given, MAST-ML
# will default to evaulating just the mean absolute error. A complete list of 
# metrics can be obtained from calling Metrics()._metric_zoo() in metrics.py.
metrics = ['r2_score', 'mean_absolute_error', 'root_mean_squared_error', 'rmse_over_stdev']


In [16]:
# As a first run, we want to define and run the case where no data split is 
# performed. This represents a full fit to all of the data. We will fit both our
# random forest and Gaussian process model, but just for the case where all
# features are used.

splitter = NoSplit()
splitter.evaluate(X=X,
                  y=y, 
                  models=models,
                  preprocessor=preprocessor,
                  selectors=[selector1],
                  metrics=metrics,
                  savepath=savepath,
                  X_extra=X_extra,
                  verbosity=3)

In [19]:
# Next, we want to do something a bit more informative. We will do a random 
# leave-out cross validation test (5-fold CV). MAST-ML will output data and plots
# for each split as well as some more comprehensive analysis performed over all
# splits. The saved model and preprocessor corresponding to the best split will
# also be put in the split directory, which can be imported for use in future
# predictions 

splitter = SklearnDataSplitter(splitter='RepeatedKFold', n_repeats=1, n_splits=5)
splitter.evaluate(X=X,
                  y=y, 
                  models=models,
                  preprocessor=preprocessor,
                  selectors=[selector2],
                  metrics=metrics,
                  plots=['Scatter', 'Histogram'],
                  savepath=savepath,
                  X_extra=X_extra,
                  verbosity=3)

In [20]:
# Randomly leaving out data isn't the only way to assess model performance. This
# dataset has logical groupings based on which element is the host. So, we can
# do another cross validation test where this time we leave out each host once.
# Let's test just our random forest model with the leave out group CV method.

splitter = SklearnDataSplitter(splitter='LeaveOneGroupOut')
splitter.evaluate(X=X,
                  y=y, 
                  models=[model1],
                  preprocessor=preprocessor,
                  groups=groups,
                  selectors=[selector2],
                  metrics=metrics,
                  plots=['Scatter', 'Histogram'],
                  savepath=savepath,
                  X_extra=X_extra,
                  verbosity=3)

In [21]:
# Now lets use just random forest and the full feature set to do random leave out CV, 
# but let's do a nested variant so we can recalibrate the predicted random forest errors.
# This will allow us to more deeply assess the uncertainty estimates of the 
# random forest model, and see that they would benefit from a correction scheme,
# which in this case is applied following the method of Palmer et al.
#
# Note that this will take longer than previous runs since we are doing nested
# CV. Since we are doing 2 repeats of leave out 1/3 of the data (n_splits=3), and 
# nesting is turned on, means we have (2*3)^2 = 36 total splits

splitter = SklearnDataSplitter(splitter='RepeatedKFold', n_repeats=2, n_splits=3)
splitter.evaluate(X=X,
                  y=y, 
                  models=[model1],
                  preprocessor=preprocessor,
                  selectors=[selector1],
                  metrics=metrics,
                  plots=['Error'],
                  savepath=savepath,
                  X_extra=X_extra,
                  nested_CV=True,
                  error_method='stdev_weak_learners', #stdev_weak_learners, jackknife_after_bootstrap
                  recalibrate_errors=True,
                  verbosity=2)

NLL optimization successful!
NLL optimization successful!
NLL optimization successful!
NLL optimization successful!
NLL optimization successful!
NLL optimization successful!
NLL optimization successful!
NLL optimization successful!
NLL optimization successful!
NLL optimization successful!
NLL optimization successful!
NLL optimization successful!
